In [1]:
import pandas as pd
import numpy as np

In [2]:
def print_group(group):
    for key, item in group:
        print(key)
        print(group.get_group(key))

In [3]:
def get_new_rate_perday(data, day, key):
    tot = data[data["day"] <= day][key].unique().shape[0]
    before = data[data["day"] < day][key].unique().shape[0]
    cur = data[data["day"] == day][key].unique().shape[0]
    old = cur + before - tot
    print("day {0}: num of ".format(day)+key+" is {0} new rate is {1}".format(cur, (cur-old)/cur))

In [4]:
test = pd.read_csv("../../IJCAI2018-backup/data/raw_data/round1_ijcai_18_test_a_20180301.zip", header=0, sep=" ")

In [5]:
train = pd.read_csv("../../IJCAI2018-backup/data/raw_data/round1_ijcai_18_train_20180301.zip", compression="zip", header=0, sep=" ")

In [6]:
labels = train.loc[:]["is_trade"]

In [7]:
print("trade rate is: {0}".format(labels[labels==1].shape[0]/labels.shape[0]))

trade rate is: 0.01886693799697995


In [8]:
t = pd.to_datetime(test['context_timestamp'], unit='s')
test["hour"] = t.dt.hour
test["day"] = t.dt.day
t = pd.to_datetime(train['context_timestamp'], unit='s')
train["hour"] = t.dt.hour
train["day"] = t.dt.day

In [9]:
data = pd.concat([train, test])

In [10]:
icl = data["item_category_list"].str.split(pat=";", expand=True)

In [11]:
data["item_category_id"] = icl[1]

In [12]:
# try to do 2-bag feats
cat_ = []
user = ["user_id", "user_gender_id", "user_occupation_id", "user_age_level"]
context = ["day", "hour", "context_page_id"]
item = ["item_id", "item_brand_id", "item_city_id", "item_category_id"]
shop = ["shop_id"]
cat_.append(user)
cat_.append(context)
cat_.append(item)
cat_.append(shop)
print(cat_)

[['user_id', 'user_gender_id', 'user_occupation_id', 'user_age_level'], ['day', 'hour', 'context_page_id'], ['item_id', 'item_brand_id', 'item_city_id', 'item_category_id'], ['shop_id']]


从 categorical feats 提取 counts

In [13]:
for lst in cat_:
    for cat_feat in lst:
        data["count_"+cat_feat] = data.groupby(cat_feat)[cat_feat].transform("size")

In [14]:
data.keys()

Index(['context_id', 'context_page_id', 'context_timestamp', 'day', 'hour',
       'instance_id', 'is_trade', 'item_brand_id', 'item_category_list',
       'item_city_id', 'item_collected_level', 'item_id', 'item_price_level',
       'item_property_list', 'item_pv_level', 'item_sales_level',
       'predict_category_property', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_delivery',
       'shop_score_description', 'shop_score_service', 'shop_star_level',
       'user_age_level', 'user_gender_id', 'user_id', 'user_occupation_id',
       'user_star_level', 'item_category_id', 'count_user_id',
       'count_user_gender_id', 'count_user_occupation_id',
       'count_user_age_level', 'count_day', 'count_hour',
       'count_context_page_id', 'count_item_id', 'count_item_brand_id',
       'count_item_city_id', 'count_item_category_id', 'count_shop_id'],
      dtype='object')

从同一大类两两一组的 categorical feats 中提取 bag feats 

In [18]:
import itertools

In [23]:
bag_feats_list = []
for name1, name2 in itertools.combinations(context, 2):
    name = "bag_"+name1+"_"+name2
    bag_feats_list.append(name)
    data[name] = data.groupby([name1, name2])["instance_id"].transform("size")

for name1, name2 in itertools.combinations(item, 2):
    name = "bag_"+name1+"_"+name2
    bag_feats_list.append(name)
    data[name] = data.groupby([name1, name2])["instance_id"].transform("size")

从不同的两类中提取 bag feats

In [26]:
for l1, l2 in itertools.combinations(cat_, 2):
    for name1, name2 in itertools.product(l1, l2):
        name = "bag_"+name1+"_"+name2
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2])["instance_id"].transform("size")

In [33]:
for l1, l2, l3 in itertools.combinations(cat_, 3):
    for name1, name2, name3 in itertools.product(l1, l2, l3):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")

In [35]:
for name1, name2, name3 in itertools.combinations(context, 3):
    name = "bag_"+name1+"_"+name2+"_"+name3
    bag_feats_list.append(name)
    data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")

for name1, name2, name3 in itertools.combinations(item, 3):
    name = "bag_"+name1+"_"+name2+"_"+name3
    bag_feats_list.append(name)
    data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")

In [36]:
for name1 in user:
    for name2, name3 in itertools.combinations(context, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")
        
for name1 in user:
    for name2, name3 in itertools.combinations(item, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")
        
for name1 in shop:
    for name2, name3 in itertools.combinations(context, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")
        
for name1 in shop:
    for name2, name3 in itertools.combinations(item, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")
        
for name1 in item:
    for name2, name3 in itertools.combinations(context, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")
        
for name1 in context:
    for name2, name3 in itertools.combinations(item, 2):
        name = "bag_"+name1+"_"+name2+"_"+name3
        bag_feats_list.append(name)
        data[name] = data.groupby([name1, name2, name3])["instance_id"].transform("size")

In [41]:
count_feats_list = []
for lst in cat_:
    for cat_feat in lst:
        count_feats_list.append("count_"+cat_feat)

In [59]:
_train = data.iloc[:train.shape[0], :]
_valid = _train[_train["day"]==24]
_train = _train[_train["day"]<24]

In [70]:
_test = data.iloc[train.shape[0]:, :]
_test = _test.drop("is_trade", axis=1)

In [84]:
bags_train_np = np.asarray(_train[bag_feats_list].apply(pd.to_numeric))
counts_train_np = np.asarray(_train[count_feats_list].apply(pd.to_numeric))
dense_train_np = np.asarray(_train[dense_list].apply(pd.to_numeric))

In [80]:
dense_list=['item_price_level',
                             'item_sales_level', 
                             'item_collected_level',
                             'item_pv_level', 
                             'user_age_level', 
                             'user_star_level',
                             'shop_review_num_level', 
                             'shop_star_level',
                             'day',
                             'hour',
                             'context_page_id',
                             'shop_review_positive_rate',
                             'shop_score_service',
                             'shop_score_delivery',
                             'shop_score_description']

In [85]:
bags_valid_np = np.asarray(_valid[bag_feats_list].apply(pd.to_numeric))
counts_valid_np = np.asarray(_valid[count_feats_list].apply(pd.to_numeric))
dense_valid_np = np.asarray(_valid[dense_list].apply(pd.to_numeric))

In [86]:
bags_test_np = np.asarray(_test[bag_feats_list].apply(pd.to_numeric))
counts_test_np = np.asarray(_test[count_feats_list].apply(pd.to_numeric))
dense_test_np = np.asarray(_test[dense_list].apply(pd.to_numeric))

In [93]:
valid_np = np.concatenate([bags_valid_np, counts_valid_np, dense_valid_np], 1)
np.save("../../IJCAI2018/prep_bag/valid", valid_np)

In [92]:
train_np = np.concatenate([bags_train_np, counts_train_np, dense_train_np], 1)
np.save("../../IJCAI2018/prep_bag/train", train_np)

In [94]:
test_np = np.concatenate([bags_test_np, counts_test_np, dense_test_np], 1)
np.save("../../IJCAI2018/prep_bag/test", test_np)

In [95]:
train_labels = np.asarray(_train["is_trade"].apply(pd.to_numeric))
np.save("../../IJCAI2018/prep_bag/train_labels", train_labels)

In [96]:
valid_labels = np.asarray(_valid["is_trade"].apply(pd.to_numeric))
np.save("../../IJCAI2018/prep_bag/valid_labels", valid_labels)

In [97]:
valid_labels.shape

(52528,)

In [10]:
data["day"].unique()

array([18, 17, 20, 21, 19, 22, 23, 24, 25])

In [11]:
num_user_test = test["user_id"].unique().shape[0]
num_user_train = train["user_id"].unique().shape[0]

In [12]:
print("per user click of test: {0}, train: {1}".format(test.shape[0] / num_user_test, train.shape[0] / num_user_train))

per user click of test: 1.353495910999779, train: 2.418576183394539


观察1： train 中平均用户点击数比 test 中大很多, 可能也是因为最后一天新用户比较多，感觉影响不大

In [13]:
for key in ["user_id", "user_occupation_id", "item_id", "item_brand_id", "item_city_id", "shop_id", "context_id"]:
    for day in range(17,26): 
        get_new_rate_perday(data, day, key)

day 17: num of user_id is 3479 new rate is 1.0
day 18: num of user_id is 37225 new rate is 0.9871591672263264
day 19: num of user_id is 34063 new rate is 0.9080233684643161
day 20: num of user_id is 32985 new rate is 0.8675155373654692
day 21: num of user_id is 34170 new rate is 0.8453321627158326
day 22: num of user_id is 32603 new rate is 0.8172254087047204
day 23: num of user_id is 29767 new rate is 0.7819733261665602
day 24: num of user_id is 26246 new rate is 0.7557723081612436
day 25: num of user_id is 12577 new rate is 0.7337202830563727
day 17: num of user_occupation_id is 5 new rate is 1.0
day 18: num of user_occupation_id is 5 new rate is 0.0
day 19: num of user_occupation_id is 5 new rate is 0.0
day 20: num of user_occupation_id is 5 new rate is 0.0
day 21: num of user_occupation_id is 5 new rate is 0.0
day 22: num of user_occupation_id is 5 new rate is 0.0
day 23: num of user_occupation_id is 5 new rate is 0.0
day 24: num of user_occupation_id is 5 new rate is 0.0
day 25: n

观察2： 每天新用户出现的概率都很高, 并且出现其他新项目的概率很低， context_id 没有重复， user_id 可以换成 count 来减小维度, context_id可以扔掉

尝试做 bag feats

In [14]:
user_day_train = train.groupby(["user_id", "day"])
user_day_hour_train = train.groupby(["user_id", "day", "hour"])
user_item_train = train.groupby(["user_id", "item_id"])
user_shop_train = train.groupby(["user_id", "shop_id"])
user_brand_train = train.groupby(["user_id", "item_brand_id"])
user_context_train = train.groupby(["user_id", "context_id"])
item_context_train = train.groupby(["item_id", "context_id"])
item_page_train = train.groupby(["item_id", "context_page_id"])

In [15]:
train[user_item_train["instance_id"].transform("size") == 1].shape

(415989, 29)

In [16]:
train[user_brand_train["instance_id"].transform("size") == 1].shape

(364795, 29)

In [17]:
# try to do 2-bag feats
user = ["user_id", "user_gender_id", "user_occupation_id", "user_age_level"]
context = ["day", "hour", "context_page_id"]
item = ["item_id", "item_brand_id", "item_city_id"]
shop = ["shop_id"]

In [18]:
import itertools

In [19]:
# for keys in itertools.product(user, item):
#     name = keys[0]+"_"+keys[1]
#     train[name] = train.groupby(list(keys))["instance_id"].transform("size")
# for keys in itertools.product(user, context):
#     name = keys[0]+"_"+keys[1]
#     train[name] = train.groupby(list(keys))["instance_id"].transform("size")
# for keys in itertools.product(user, shop):
#     name = keys[0]+"_"+keys[1]
#     train[name] = train.groupby(list(keys))["instance_id"].transform("size")
for keys in itertools.product(item, context):
    name = keys[0]+"_"+keys[1]
    train[name] = train.groupby(list(keys))["instance_id"].transform("size")

In [20]:
for i in itertools.combinations(item, 2):
    print(i)

('item_id', 'item_brand_id')
('item_id', 'item_city_id')
('item_brand_id', 'item_city_id')


比较 train 和 test 的 icl

In [21]:
icl_train = train["item_category_list"].str.split(pat=";", expand=True)

In [22]:
icl_test = test["item_category_list"].str.split(pat=";", expand=True)

观察3： train 和 test 的分类目录完全相同

In [23]:
print(set(icl[0].unique()) == set(icl_test[0].unique()))
print(set(icl[1].unique()) == set(icl_test[1].unique()))
print(set(icl[2].unique()) == set(icl_test[2].unique()))

True
True
True


In [24]:
for i in icl[2].unique():
    print(i)
    print("train: {0}, trade rate: {1}".format(icl[icl[2] == i].shape[0] / icl.shape[0], labels[icl[2] == i].shape[0]/labels.shape[0]))
    print("test: {0}".format(icl_test[icl_test[2] == i].shape[0] / icl_test.shape[0]))

None
train: 0.0, trade rate: 0.0
test: 0.0
8868887661186419229
train: 0.0039862968431708, trade rate: 0.0039862968431708
test: 0.0040825213651951445
6233669177166538628
train: 0.0002572479075078743, trade rate: 0.0002572479075078743
test: 0.00021773447281040771


观察4： train 和 test 的最细层目录分类分布基本一样, 而且都特别少, 同时 trade rate 也没有贡献，可以扔掉

In [25]:
for i in icl[1].unique():
    print(i)
    print("train: {0}".format(icl[icl[1] == i].shape[0] / icl.shape[0]))
    print("test: {0}".format(icl_test[icl_test[1] == i].shape[0] / icl_test.shape[0]))

5799347067982556520
train: 0.15062387846186664
test: 0.14838604322029286
8277336076276184272
train: 0.3153461971230063
test: 0.33193620379946653
5755694407684602296
train: 0.2143648904709519
test: 0.21953078221109357
4879721024980945592
train: 0.002524375807821173
test: 0.002993849001143106
509660095530134768
train: 0.1577368876767795
test: 0.15959936857002885
2436715285093487584
train: 0.004111783627320982
test: 0.003919220510587339
7258015885215914736
train: 0.11096168888479895
test: 0.07969081704860922
8710739180200009128
train: 0.014815806315331557
test: 0.016547819933590984
2011981573061447208
train: 0.020000501947136602
test: 0.02030373958957052
22731265849056483
train: 0.00018404728342026779
test: 0.00010886723640520386
3203673979138763595
train: 0.0046973886200218345
test: 0.01213869685918023
2642175453151805566
train: 0.004243544750678675
test: 0.004300255838005552
1968056100269760729
train: 0.000389009030865566
test: 0.0005443361820260192


第二层级目录的分布也基本一样， 保留这一层目录， 可以作为普通的categorical feats 做count 和 bag

In [26]:
ipl = train["item_property_list"].str.split(pat=";", expand=True)
ipl_test = test["item_property_list"].str.split(pat=";", expand=True)

In [39]:
ipl = ipl.fillna(-1)

对 ipl 做 "n_hot"

In [27]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [67]:
ipl_ = ipl.apply(lambda x: x.str[-3:])

In [72]:
ipl_ = ipl_.fillna(-1)

In [92]:
values = np.array(ipl_.apply(pd.to_numeric))
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values.flatten())
# binary encode
onehot_encoder = OneHotEncoder(sparse=True)
integer_encoded = integer_encoded.reshape(integer_encoded.shape[0], 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [93]:
onehot_encoded.shape

(47813800, 1001)

In [94]:
ipl_one_hot = onehot_encoded[478138:]

In [95]:
ipl_one_hot.shape

(47335662, 1001)

In [97]:
data.keys()

Index(['context_id', 'context_page_id', 'context_timestamp', 'day', 'hour',
       'instance_id', 'is_trade', 'item_brand_id', 'item_category_list',
       'item_city_id', 'item_collected_level', 'item_id', 'item_price_level',
       'item_property_list', 'item_pv_level', 'item_sales_level',
       'predict_category_property', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_delivery',
       'shop_score_description', 'shop_score_service', 'shop_star_level',
       'user_age_level', 'user_gender_id', 'user_id', 'user_occupation_id',
       'user_star_level'],
      dtype='object')